### Задание

Реализуйте алгоритм классификации метод k ближайших соседей.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен

Необходимо реализовать предложенный интерфейс.
В качестве решения необходимо отправить боту, указав seminar04 в поле caption,  следующие файлы:
* knn_model.py - содержит класс, реализующий ваш алгоритм
* knn_results.txt - файл с результатами тестов, который можно будет сгенерировать командой python3 knn_tests.py

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.neighbors.KNeighborsClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [267]:
import numpy as np
import matplotlib.pyplot as plt

In [1081]:
class KnnBruteClassifier(object):
    '''Классификатор реализует взвешенное голосование по ближайшим соседям. 
    Поиск ближайшего соседа осуществляется полным перебором.
    Параметры
    ----------
    n_neighbors : int, optional
        Число ближайших соседей, учитывающихся в голосовании
    weights : str, optional (default = 'uniform')
        веса, используемые в голосовании. Возможные значения:
        - 'uniform' : все веса равны.
        - 'distance' : веса обратно пропорциональны расстоянию до классифицируемого объекта
        -  функция, которая получает на вход массив расстояний и возвращает массив весов
    metric: функция подсчета расстояния (по умолчанию l2).
    '''
    
    
        
        
    def __init__(self, n_neighbors=1, weights='uniform',*metric):
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        if metric:
            self.metric = metric
        else:
            self.metric = l2
            
    def __l2(self,x,y):
        return np.sum(((x-y)**2))**(1/2)
            
    
        
    def __fun_weights(self,dist):
        list_weights =[]
        if type(self.weights)==str:
            if self.weights == 'uniform':
                list_weights = np.ones(len(dist), dtype=np.int64)
            if self.weights == 'distance':
                list_weights = np.array([1/d for d in dist])
        if hasattr(self.weights, '__call__'):
            list_weights = weights(dist)
        return list_weights
            
            
    def _voit():
        self.weights = weights
        self.n_neighbors = n_neighbors
    
    
     
    def fit(self, x, y):
        '''Обучение модели.
        Парметры
        ----------
        x : двумерным массив признаков размера n_queries x n_features
        y : массив/список правильных меток размера n_queries
        Выход
        -------
        Метод возвращает обученную модель
        '''
        self.x_train = x
        self.y_train = y 
        
    def predict(self, x):
        """ Предсказание класса для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        y : Массив размера n_queries
        """
        res = np.array([], dtype=np.int64)
        for ob in x:
            list_dist = np.array([metric(xi,ob) for xi in self.x_train]) ## массив с расстояними от x_{i} до x_train{j}
            ind = list_dist.argsort()[:self.n_neighbors] ## индекс первых n_neighbors ближайших соседей среди x_train
            res_kNN = self.y_train[ind] ## значения y_train первых n_neighbors ближайших соседей к x_{i} среди x_train
            
            list_dist = list_dist[ind]
            list_weights = self.__fun_weights(list_dist)  ## массив с weights 
            ind = list_dist.argsort()[:self.n_neighbors] ## индекс первых n_neighbors ближайших соседей среди x_train
            voit = []
            for y in np.unique(res_kNN):
                ind_y = np.where(res_kNN == y)[0]
                sum_y = np.sum(list_weights[ind_y])
                voit.append([y,sum_y])
            voit = np.array(voit)
            res = np.append(res, int(voit[np.argmax(voit, axis=0)[1]][0]))
            
        return res
        
    def predict_proba(self, X):
        """Возвращает вероятности классов для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        p : массив размера n_queries x n_classes] c вероятностями принадлежности 
        объекта к каждому классу
        """
        res_prob = np.array([])
        for ob in X:
            list_dist = np.array([metric(xi,ob) for xi in self.x_train]) ## массив с расстояними от x_{i} до x_train{j}
            ind = list_dist.argsort()[:self.n_neighbors] ## индекс первых n_neighbors ближайших соседей среди x_train
            res_kNN = self.y_train[ind] ## значения y_train первых n_neighbors ближайших соседей к x_{i} среди x_train
            list_dist = list_dist[ind]
            list_weights = self.__fun_weights(list_dist)  ## массив с weights
            voit = []
            sum_all =np.sum(list_weights)
            for y in range(10):
                ind_y = np.where(res_kNN == y)[0]
                sum_y = np.sum(list_weights[ind_y])
                voit.append([y,sum_y/sum_all])
            voit = np.array(voit)
            voit = voit[::,1]
            res_prob = np.append(res_prob,voit)
            
        res_prob = np.reshape(res_prob, (len(X),-1))
        return res_prob

        
    def kneighbors(self, x, n_neighbors):
        """Возвращает n_neighbors ближайших соседей для всех входных объектов и расстояния до них
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        neigh_dist массив размера n_queries х n_neighbors
        расстояния до ближайших элементов
        neigh_indarray, массив размера n_queries x n_neighbors
        индексы ближайших элементов
        """
        neigh_dist = []
        neigh_indarray =[]

        for ob in x:
            list_dist = np.array([metric(xi,ob) for xi in self.x_train]) ## массив с расстояними от x_{i} до x_train{j}
            ind = list_dist.argsort()[:n_neighbors] ## индекс первых n_neighbors ближайших соседей среди x_train
            neigh_dist.append(list_dist[ind])
            neigh_indarray.append(ind)
        return np.array(neigh_dist), np.array(neigh_indarray)
        

In [1096]:
def load_file(filename):
    """
    TODO: Необходимо загрузить файл задания и вернуть словарь с ключами "X_train", "X_test", "y_train"
    """
    return  np.load(filename,allow_pickle=True).item()


In [1097]:
input_filename = "knn_data_085.npy" #TODO задать путь к входному файлу
data_dict = load_file(input_filename)

In [1098]:
model = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_n5_y_predict = model.predict(data_dict["X_test"])

In [1099]:
model = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_10_y_predict = model.predict(data_dict["X_test"])

In [1100]:
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_distance_n5_y_predict = model.predict(data_dict["X_test"])

In [1101]:
output_filename = "knn_results.npy"
result_dict = {
    "input_filename": input_filename,
    "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
    "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
    "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
}
np.save(output_filename, result_dict)